In [ ]:
import streamlit as st
import pandas as pd
import pickle  # Assuming model is saved as a pickle file

# Load precomputed taxi time data
taxi_times = pd.read_csv("taxi_times_lookup.csv")

# Load trained model
with open("flight_delay_model.pkl", "rb") as file:
    model = pickle.load(file)

def get_taxi_times(airline, origin, destination):
    """Fetch precomputed average TaxiIn, TaxiOut, WheelsOn, WheelsOff times."""
    match = taxi_times[(taxi_times['Airline'] == airline) & 
                       (taxi_times['Origin'] == origin) & 
                       (taxi_times['Destination'] == destination)]
    
    if not match.empty:
        return match.iloc[0]['TaxiIn'], match.iloc[0]['TaxiOut'], match.iloc[0]['WheelsOn'], match.iloc[0]['WheelsOff']
    else:
        return 0, 0, 0, 0  # Default values if no match found

# Streamlit UI
st.title("Flight Delay Prediction")

# User inputs
airline = st.selectbox("Select Airline", taxi_times["Airline"].unique())
origin = st.selectbox("Select Origin", taxi_times["Origin"].unique())
destination = st.selectbox("Select Destination", taxi_times["Destination"].unique())
departure_time = st.time_input("Departure Time")
date = st.date_input("Flight Date")

# Get estimated values
taxi_in, taxi_out, wheels_on, wheels_off = get_taxi_times(airline, origin, destination)

# Prepare input for model
input_data = pd.DataFrame({
    "Airline": [airline],
    "Origin": [origin],
    "Destination": [destination],
    "DepTime": [departure_time.hour * 100 + departure_time.minute],  # Convert time to HHMM format
    "Date": [date.strftime("%Y-%m-%d")],  # Convert date
    "TaxiIn": [taxi_in],
    "TaxiOut": [taxi_out],
    "WheelsOn": [wheels_on],
    "WheelsOff": [wheels_off]
})

# Make prediction
if st.button("Predict Delay"):
    prediction = model.predict(input_data)
    st.write(f"Predicted Delay: {prediction[0]} minutes")
